In [41]:
!pip install streamlit streamlit-option-menu

In [50]:
%%writefile app.py

import sqlite3
import requests
import pandas as pd
import streamlit as st
from streamlit_option_menu import option_menu

conn = sqlite3.connect("mydb.db")
cursor = conn.cursor()

api_key = "1a7ae53e-a8d5-4ca8-8cbf-e12a843ceec9"
url = "https://api.harvardartmuseums.org/object"

#------------------------------------------------ Create 3 SQL tables--------------------------------------------------------------------------------------#


def create_tables():
    cursor.execute("""create table if not exists artifacts_metadata (
                    id integer primary key,
                    title text,
                    culture text,
                    dated text,
                    period text,
                    division text,
                    medium text,
                    dimensions varchar(75),
                    dept text,
                    description text,
                    classification text,
                    accessionyear integer,
                    accessmethod text
)""")
    cursor.execute("""create table if not exists artifacts_media(
                    objectid integer,
                    imagecount int,
                    mediacount int,
                    colorcount int,
                    rank int,
                    datebegin int,
                    dateend int,
                    FOREIGN KEY(objectid)REFERENCES artifacts_metadata(id)

)""")
    cursor.execute("""create table if not exists artifacts_colors(
                  objectid integer,
                  color text,
                  spectrum text,
                  hue text,
                  percent REAL,
                  css text,
                  FOREIGN KEY(objectid)REFERENCES artifacts_metadata(id)
)""")


create_tables()

# ------------------------------------------------------------------------Data Collection per class-----------------------------------------------------------------#

def classes(api_key,class_name):
    all_records = []


    for page in range(1, 26):
        params = {
            "apikey": api_key,
            "size": 100,
            "page": page,
            "hasimage": 1,
            "classification": class_name
        }

        response = requests.get(url, params=params)

        data = response.json()
        records = data.get('records', [])
        all_records.extend(records)

        return all_records

# Collecting metadata, media and color details

def artifacts_details(records):

      artifacts = []
      media = []
      colors = []

      for i in records:
          artifacts.append(dict(
              id = i['id'],
              title = i['title'],
              culture = i['culture'],
              dated = i['dated'],
              period = i.get('period'),
              division = i['division'],
              medium = i.get('medium'),
              dimensions = i.get("dimensions"),
              department = i.get("department"),
              description = i.get('description'),
              classification = i['classification'],
              accessionyear = i['accessionyear'],
              accessionmethod = i['accessionmethod']
                          ))

          media.append(dict(
              objectid = i['objectid'],
              imagecount = i['imagecount'],
              mediacount = i['mediacount'],
              colorcount = i['colorcount'],
              rank = i['rank'],
              datebegin = i['datebegin'],
              dateend = i['dateend']

          ))

          sub_list = i['colors']
          for j in sub_list:
              colors.append(dict(
                  objectid = i['objectid'],
                  color = j.get('color'),
                  spectrum= j.get('spectrum'),
                  hue = j['hue'],
                  percent = j['percentage'],
                  css3 = j['css3']

              ))


      return artifacts,media,colors

# ----------------------------------------------------------------------------------Data Insertion -------------------------------------------------------------------#

def insert_values(arti,med,col):
      insert_meta = """insert into artifacts_metadata values(?,?,?,?,?,?,?,?,?,?,?,?,?)"""
      insert_media = """insert into artifacts_media values(?,?,?,?,?,?,?)"""
      insert_col  = """insert into artifacts_colors values(?,?,?,?,?,?)"""

      for i in arti:
          values1 = (i['id'], i['title'], i['culture'],i['dated'], i['period'], i['division'], i['medium'], i['dimensions'], i['department'], i['description'], i['classification'], i['accessionyear'], i['accessionmethod'])
          cursor.execute(insert_meta,values1)

      for i in med:
          values2 = (i['objectid'], i['imagecount'], i['mediacount'], i['colorcount'], i['rank'], i['datebegin'], i['dateend'])
          cursor.execute(insert_media,values2)

      for i in col:
          values3 = (i['objectid'], i['color'], i['spectrum'], i['hue'], i['percentage'], i['css3'])
          cursor.execute(insert_col,values3)

      conn.commit()

#---------------------------------------------------------------------------- Streamlit UI ----------------------------------------------------------------------------#

st.set_page_config(layout="wide")



st.markdown("<h1 style='text-align: center; color: black;'>🎨🏛️ Harvard’s Artifacts Collection</h1>", unsafe_allow_html=True)




classification = st.text_input("Enter a classification:") # Coins
button = st.button("Collect data")
menu = option_menu(None,["Select Your Choice","Migrate to SQL","SQL Queries"], orientation="horizontal")




if button:
    if classification != '':
        records = classes(api_key,classification)
        arti ,med,col = artifacts_details(records)
        c1,c2,c3 = st.columns(3)
        with c1:
              st.header("Metadata")
              st.json(arti)
        with c2:
              st.header("Media")
              st.json(med)
        with c3:
              st.header("Colours")
              st.json(col)

    else:
          st.error("Kindly enter a classification")



if menu == 'Migrate to SQL':

        cursor.execute("select distinct(classification) from artifacts_metadata")
        result = cursor.fetchall()
        classes_list = [i[0] for i in result]


        st.subheader("Insert the collected data")
        if st.button("Insert"):
          if classification not in classes_list:

                records = classes(api_key,classification)
                arti,med,col = artifacts_details(records)
                insert_values(arti,med,col)
                st.success("Data Inserted successfully")

                st.header("Inserted Data:")
                st.divider()

                st.subheader("Artifacts Metadata")
                cursor.execute("select * from artifacts_metadata")
                result1 = cursor.fetchall()
                columns = [i[0] for i in cursor.description]
                df1 = pd.DataFrame(result1,columns=columns)
                st.dataframe(df1)

                st.subheader("Artifacts Media")
                cursor.execute("select * from artifacts_media")
                result2 = cursor.fetchall()
                columns = [i[0] for i in cursor.description]
                df2 = pd.DataFrame(result2,columns=columns)
                st.dataframe(df2)

                st.subheader("Artifacts Colors")
                cursor.execute("select * from artifacts_colors")
                result3 = cursor.fetchall()
                columns = [i[0] for i in cursor.description]
                df3 = pd.DataFrame(result3,columns=columns)
                st.dataframe(df3)
          else:
            st.error("Classification already exists!! Kindly try a different class ! ")




elif menu == "SQL Queries":

        option = st.selectbox("Queries",
                ("1.List all artifacts from the 11th century belonging to Byzantine culture.",
                  "2.What are the unique cultures represented in the artifacts?",
                  "3.List all artifacts from the Archaic Period",
                  "4.List artifact titles ordered by accession year in descending order.",
                  "5.How many artifacts are there per department?",
                  "6.Which artifacts have more than 3 imagecount?",
                  "7.What is the average rank of all artifacts?",
                  "8.Which artifacts have a higher colorcount than mediacount?",
                  "9.List all artifacts created between 1500 and 1600.",
                  "10.How many artifacts have no media files?",
                  "11.What are all the distinct hues used in the dataset?",
                  "12.What are the top 5 most used colors by frequency?",
                  "13.What is the average coverage percentage for each hue?",
                  "14.List all colors used for a given artifact ID.",
                  "15.What is the total number of color entries in the dataset?",
                  "16.List artifact titles and hues for all artifacts belonging to the Byzantine culture.",
                  "17.List each artifact title with its associated hues.",
                  "18.Get artifact titles, cultures, and media ranks where the period is not null.",
                  "19.Find artifact titles ranked in the top 10 that include the color hue 'Grey'.",
                  "20.How many artifacts exist per classification, and what is the average media count for each?",
                  "21.Get artifact distinct classifications and medium where the color hue 'Blue'.",
                  "22.List artifact titles between 1200 to 1300.",
                  "23.How many artifacts exist per classification with image count.",
                  "24.List artifact titles with colors ranked in the top 5.",
                  "25.List artifact titles and colorcount for all artifacts belonging to the Flemish culture."),index =None,placeholder="Select a query")




        if option == "1.List all artifacts from the 11th century belonging to Byzantine culture.":
            cursor.execute("""select * from artifacts_metadata where dated = '11th century' and culture = 'Byzantine' """)
            result = cursor.fetchall()
            df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
            st.dataframe(df)



        elif option == "2.What are the unique cultures represented in the artifacts?":
           cursor.execute("""select distinct(culture) from artifacts_metadata""")
           result = cursor.fetchall()
           df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
           st.dataframe(df)

        elif option == "3.List all artifacts from the Archaic Period":
           cursor.execute("""select * from artifacts_metadata where period = 'Archaic' """)
           result = cursor.fetchall()
           df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
           st.dataframe(df)

        elif option == "4.List artifact titles ordered by accession year in descending order.":
           cursor.execute("""select title from artifacts_metadata order by accession_year desc""")
           result = cursor.fetchall()
           df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
           st.dataframe(df)

        elif option == "5.How many artifacts are there per department?":
           cursor.execute("""select dept,count(*) from artifacts_metadata group by dept""")
           result = cursor.fetchall()
           df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
           st.dataframe(df)

        elif option == "6.Which artifacts have more than 3 imagecount?":
           cursor.execute("select * from artifacts_media where imagecount > 3")
           result = cursor.fetchall()
           df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
           st.dataframe(df)

        elif option == "7.What is the average rank of all artifacts?":
          cursor.execute("select avg(rank) from artifacts_media")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "8.Which artifacts have a higher colorcount than mediacount?":
          cursor.execute("select * from artifacts_media where mediacount > colorcount")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "9.List all artifacts created between 1500 and 1600.":
          cursor.execute("select * from artifacts_media where datebegin between 1500 and 1600")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "10.How many artifacts have no media files?":
          cursor.execute("select * from artifacts_media where mediacount=0")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "11.What are all the distinct hues used in the dataset?":
          cursor.execute("select DISTINCT hue from artifacts_colors")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "12.What are the top 5 most used colors by frequency?":
          cursor.execute("select hue, count(*) as count from artifacts_colors group by hue order by count desc limit 5")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "13.What is the average coverage percentage for each hue?":
          cursor.execute("select hue, avg(percentage) from artifacts_colors group by hue")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "14.List all colors used for a given artifact ID.":
          cursor.execute("select objectid,color from artifacts_colors group by objectid")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "15.What is the total number of color entries in the dataset?":
          cursor.execute("select count(DISTINCT color) from artifacts_colors")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "16.List artifact titles and hues for all artifacts belonging to the Byzantine culture.":
          cursor.execute("""select metadata.title, metadata.culture, colors.hue
                        from artifacts_colors colors join artifacts_metadata metadata
                        on metadata.id=colors.objectid where metadata.culture='Byzantine'""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "17.List each artifact title with its associated hues.":
          cursor.execute("""select metadata.title, colors.hue
                        from artifacts_metadata metadata join artifacts_colors colors
                        on metadata.id=colors.objectid""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "18.Get artifact titles, cultures, and media ranks where the period is not null.":
          cursor.execute("""select metadata.title, metadata.culture, metadata.period, media.rank
                        from artifacts_media media join artifacts_metadata metadata
                        on metadata.id=media.objectid where metadata.period is not null""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "19.Find artifact titles ranked in the top 10 that include the color hue 'Grey'.":
          cursor.execute("""select metadata.title, media.rank, colors.hue
                        from artifacts_media media join artifacts_metadata metadata
                        on media.objectid=metadata.id join artifacts_colors colors
                        on metadata.id=colors.objectid where colors.hue='Grey' order by media.rank desc limit 10""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "20.How many artifacts exist per classification, and what is the average media count for each?":
          cursor.execute("""select metadata.classification, avg(media.mediacount) as mediacount
                        from artifacts_metadata metadata join artifacts_media media
                        on metadata.id=media.objectid group by metadata.classification order by avg(media.mediacount)""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "21.Get artifact distinct classifications and medium where the color hue 'Blue'.":
          cursor.execute("""select DISTINCT metadata.classification, metadata.medium, colors.hue
                        from artifacts_metadata metadata join artifacts_colors colors
                        on metadata.id=colors.objectid where colors.hue='Blue' group by metadata.classification""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "22.List artifact titles between 1200 to 1300.":
          cursor.execute("""select metadata.title, media.datebegin
                        from artifacts_metadata metadata join artifacts_media media
                        on metadata.id=media.objectid between 1550 and 1600 """)
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "23.How many artifacts exist per classification with image count.":
          cursor.execute("""select metadata.classification, media.imagecount
                        from artifacts_metadata metadata join artifacts_media media
                        on metadata.id=media.objectid group by metadata.classification order by media.imagecount""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "24.List artifact titles with colors ranked in the top 5.":
          cursor.execute("""select metadata.title, media.rank, colors.color
                        from artifacts_metadata metadata join artifacts_media media
                        on metadata.id=media.objectid join artifacts_colors colors
                        on metadata.id=colors.objectid order by media.rank desc limit 5""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)

        elif option == "25.List artifact titles and colorcount for all artifacts belonging to the Flemish culture.":
          cursor.execute("""select metadata.title, metadata.culture, media.colorcount
                        from artifacts_metadata metadata join artifacts_media media
                        on metadata.id=media.objectid where metadata.culture='Flemish'""")
          result = cursor.fetchall()
          df = pd.DataFrame(result,columns = [i[0] for i in cursor.description])
          st.dataframe(df)


conn.close()


Overwriting app.py


In [47]:
# @title Setup code
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
import subprocess
subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

--2025-08-30 07:32:55--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.8.1/cloudflared-linux-amd64 [following]
--2025-08-30 07:32:55--  https://github.com/cloudflare/cloudflared/releases/download/2025.8.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/0593a2fc-d1d2-46e1-aaac-c8f6bd891eca?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-30T08%3A11%3A52Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-30T0

In [48]:
!streamlit run /content/app.py &>/content/logs.txt &

In [49]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://thomas-databases-samoa-sullivan.trycloudflare.com
